In [1]:
from PIL import Image
import glob

def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 128*128 사이즈로 바꾸어 저장합니다.
	target_size=(224,224)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

In [2]:
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\scissors"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

0  images to be resized.
0  images resized.
가위 이미지 resize 완료!


In [3]:
# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")

0  images to be resized.
0  images resized.
바위 이미지 resize 완료!


In [4]:
# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\paper"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

0  images to be resized.
0  images resized.
보 이미지 resize 완료!


In [5]:
# !pip install matplotlib
# !pip install scikit-learn
# !pip install scipy

In [6]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

def load_data(img_path, number_of_data=7491):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=224
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.float32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.float32)

    idx=0
    for file in glob.iglob(img_path+'/scissors/*.jpg'):
        img = Image.open(file).resize((img_size, img_size))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.float32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = Image.open(file).resize((img_size, img_size))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.float32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = Image.open(file).resize((img_size, img_size))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.float32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = "dataset" #폴더명
(X, y)=load_data(image_dir_path)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=0, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.4, random_state=0, stratify=y_val)

x_train_norm = X_train/255.0   # 입력은 0~1 사이의 값으로 정규화
x_val_norm = X_val/255.0
print("x_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 7491 입니다.
x_train shape: (4494, 224, 224, 3)
y_train shape: (4494,)


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow import keras

# 사전 학습된 ResNet50 모델을 불러오고 마지막 분류 레이어를 제외합니다.
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # 사전 학습된 가중치를 고정합니다.

# 새로운 모델 구성
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # 평탄화 작업
    layers.Dense(128, activation='relu'),  # 추가 레이어
    layers.Dropout(0.5),  # 과적합 방지를 위한 드롭아웃
    layers.Dense(3, activation='softmax')  # 가위, 바위, 보 3개 클래스를 위한 출력 레이어
])


In [8]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers, models
# import numpy as np

# model = models.Sequential()
# model.add(layers.Input(shape=(128, 128, 3)))  # 입력 크기 지정
# model.add(layers.Conv2D(32, (3, 3), activation='relu'))
# model.add(keras.layers.MaxPool2D(2,2))
# model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.MaxPooling2D((2,2)))
# model.add(keras.layers.Dropout(0.5))
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)))
# model.add(keras.layers.Dropout(0.5))
# model.add(keras.layers.Dense(3, activation='softmax'))
# # 😎쌤Tip.
# # pretrained model을 써서 가위, 바위, 보 데이터셋을 가지고 fine-tune 파인튜닝 하고 싶어.
# # timeframe, temporal modeling
# model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time
import datetime

optimizer = keras.optimizers.Adam(learning_rate=0.00001)

model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# ModelCheckpoint 콜백 설정
checkpoint = ModelCheckpoint(
    'dataset/model/best_model012.keras',  # 경로를 지정하여 저장
    monitor='val_loss',
    save_best_only=True,
    mode='min'
)

# 데이터 증강
start = time.time()
datagen = ImageDataGenerator(
    rotation_range=20,     # 이미지 회전
    width_shift_range=0.2, # 가로로 이동
    height_shift_range=0.2,# 세로로 이동
    shear_range=0.2,       # 시야각 변환
    zoom_range=0.2,        # 확대/축소
    horizontal_flip=True,  # 좌우 반전
    fill_mode='nearest'
)

# 증강된 이미지로 모델 학습
model.fit(datagen.flow(x_train_norm, y_train),
            validation_data=(x_val_norm, y_val),
            epochs=100,
            callbacks=[checkpoint])   # checkpoint 콜백 추가
sec = time.time() - start
act_time = str(datetime.timedelta(seconds=sec))
print(act_time)

# # 모델 학습
# start = time.time()
# model.fit(x_train_norm, y_train,
#             validation_data=(x_val_norm, y_val),
#             epochs=100,
#             callbacks=[checkpoint])   # checkpoint 콜백 추가
# sec = time.time() - start
# act_time = str(datetime.timedelta(seconds=sec))
# print(act_time)

c:\Users\zqrc0\anaconda3\envs\motionpy310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 114s 784ms/step - accuracy: 0.3378 - loss: 1.4107 - val_accuracy: 0.3309 - val_loss: 1.1182
Epoch 2/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 110s 779ms/step - accuracy: 0.3461 - loss: 1.2460 - val_accuracy: 0.3331 - val_loss: 1.1011
Epoch 3/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 108s 770ms/step - accuracy: 0.3303 - loss: 1.2442 - val_accuracy: 0.3565 - val_loss: 1.0992
Epoch 4/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 107s 761ms/step - accuracy: 0.3452 - loss: 1.2032 - val_accuracy: 0.3404 - val_loss: 1.0976
Epoch 5/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 109s 771ms/step - accuracy: 0.3203 - loss: 1.1991 - val_accuracy: 0.3526 - val_loss: 1.0963
Epoch 6/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 106s 752ms/step - accuracy: 0.3433 - loss: 1.1744 - val_accuracy: 0.3459 - val_loss: 1.0951
Epoch 7/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 106s 753ms/step - accuracy: 0.3163 - loss: 1.1741 - val_accuracy: 0.3615 - val_loss: 1.0940
Epoch 8/100
141/141 ━━━━━━━━━━━━━━━━━━━━ 107s 758ms/step - accuracy: 

In [ ]:
import glob

# image_dir_path = "dataset/test"
# (x_test, y_test)=load_data(X_test, X_test.shape[0])
x_test_norm = X_test/255.0

print("x_test shape : {}".format(X_test.shape))
print("y_test shape : {}".format(y_test.shape))

x_test shape : (1199, 224, 224, 3)
y_test shape : (1199,)


In [ ]:
from tensorflow import keras
# 저장된 최적의 모델 로드
best_model = keras.models.load_model('dataset/model/best_model012.keras')

# 테스트 데이터로 평가
test_loss, test_accuracy = best_model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

38/38 - 21s - 564ms/step - accuracy: 0.5630 - loss: 1.0153
test_loss : 1.0153329372406006
test_accuracy : 0.5629691481590271


In [ ]:
# 테스트 데이터로 평가
test_loss, test_accuracy = best_model.evaluate(x_val_norm, y_val, verbose=2)
print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

57/57 - 31s - 541ms/step - accuracy: 0.5473 - loss: 1.0152
test_loss : 1.0152113437652588
test_accuracy : 0.5472747683525085


In [ ]:
predicted_result = best_model.predict(x_test_norm)	# model이 추론한 확률값
predicted_labels = np.argmax(predicted_result, axis=1)

idx=100		# 값을 변경해서 찾아보자
print('model.predict() 결과 : ', predicted_result[idx])
print('model이 추론한 가장 가능성이 높은 결과 : ', predicted_labels[idx])
print('실제 데이터의 라벨 : ', y_test[idx])

38/38 ━━━━━━━━━━━━━━━━━━━━ 22s 554ms/step
model.predict() 결과 :  [0.33270252 0.34016725 0.32713023]
model이 추론한 가장 가능성이 높은 결과 :  1
실제 데이터의 라벨 :  2.0


In [ ]:
from sklearn.metrics import classification_report

# 실제 레이블을 y_test에 저장했다고 가정
# 모델 예측
predicted_result = best_model.predict(x_val_norm)
predicted_labels = np.argmax(predicted_result, axis=1)

# 클래스별 f1 score와 accuracy 출력
print(classification_report(y_val, predicted_labels))

57/57 ━━━━━━━━━━━━━━━━━━━━ 30s 526ms/step
              precision    recall  f1-score   support

         0.0       0.49      0.79      0.61       634
         1.0       0.59      0.47      0.52       566
         2.0       0.67      0.36      0.47       598

    accuracy                           0.55      1798
   macro avg       0.58      0.54      0.53      1798
weighted avg       0.58      0.55      0.53      1798



In [ ]:
from sklearn.metrics import classification_report

# 실제 레이블을 y_test에 저장했다고 가정
# 모델 예측
predicted_result = best_model.predict(x_val_norm)
predicted_labels = np.argmax(predicted_result, axis=1)

# 클래스별 f1 score와 accuracy 출력
print(classification_report(y_val, predicted_labels))

57/57 ━━━━━━━━━━━━━━━━━━━━ 30s 531ms/step
              precision    recall  f1-score   support

         0.0       0.49      0.79      0.61       634
         1.0       0.59      0.47      0.52       566
         2.0       0.67      0.36      0.47       598

    accuracy                           0.55      1798
   macro avg       0.58      0.54      0.53      1798
weighted avg       0.58      0.55      0.53      1798



In [ ]:
# import random
# wrong_predict_list=[]
# for i, _ in enumerate(predicted_labels):
#     if predicted_labels[i] != y_test[i]:
#         wrong_predict_list.append(i)
        
# samples = random.choices(population=wrong_predict_list, k=10) # k값을 수정하면 더 많은 값의 수를 볼 수 있다.

# for n in samples:
#     print("예측확률분포: " + str(predicted_result[n]))
#     print("라벨: " + str(y_test[n]) + ", 예측결과: " + str(predicted_labels[n]))
#     # plt.imshow(x_test[n], cmap=plt.cm.binary)
#     print(n)
#     # plt.show()

In [ ]:
# model.save('dataset/model/model.keras', include_optimizer=False)